# Circuit Connectome

We develop an analysis for the connectome of the mouse SSCx circuit.
The analysis that we will develop can be applied to any SONATA circuit 
loaded with BluePySnap

In [1]:
#top level imports
import os
from collections import OrderedDict
import pandas as pd
from tqdm import tqdm
from dmt.tk.plotting import Bars, LinePlot, Crosses, MultiPlot
from dmt.data.observation import measurement
from dmt.tk.phenomenon import Phenomenon
from dmt.tk.parameters import Parameters
from neuro_dmt import terminology
from dmt.tk.utils import datasets
from dmt.data.observation import SampleMeasurement, Summary
from neuro_dmt.utils.geometry.roi import Cuboid
from neuro_dmt.analysis.reporting import CircuitProvenance, CheetahReporter
from neuro_dmt.analysis.circuit import BrainCircuitAnalysis

use_mock = False

## Mock Circuit

We have implemented a mock circuit that can be used (by setting variable 
`use_mock=True`.

In [2]:
path_circuit = os.path.join(
    "/gpfs/bbp.cscs.ch/project/proj66/circuits/O1/20200330/sonata")

## SONATA circuit model and adapter

We have defined classes to work with SONATA circutis.

In [3]:
from neuro_dmt.library.models.sonata.circuit.model import\
    SonataCircuitModel
from neuro_dmt.library.models.sonata.circuit.adapter import\
    SonataCircuitAdapter

circuit_model =\
    circuit_mock() if use_mock else\
    SonataCircuitModel(path_circuit_data=path_circuit)
adapter = SonataCircuitAdapter()

## Reporter

Our analyses will produce a report object that can be posted as HTMLs using a reporter.

In [4]:
from neuro_dmt.analysis.reporting import CheetahReporter
reporter = CheetahReporter(
    path_output_folder=os.path.join(os.getcwd(), "analyses"))

For a connectome, we will be interested in number and strength of the connections
a node (*i.e.* a cell). There is a connection from node $X$ to $Y$ if there is
atleast one synapse from $X$ to $Y$. The strength of this connection is the 
number of synapses connecting $X$ to $Y$. 

Synaptic connections are directed. Thus if $X$ synapses onto $Y$, then $X\rightarrow Y$ is
an efferent connection of $X$ and an afferent connection of $Y$. 

Next we analyze statistical properties of cell connections.

# Node Degree


How many incoming (*afferent*) or outgoing (*efferent*) connections does a cell
have?


In [5]:
from dmt.model.interface import Interface
class ConnectomeAdapterInterface(Interface):
    pass